In [1]:
# useful starting line 
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
cd ../

/home/mbanga/Documents/Epfl/ml_revisions


# Load Data

In [3]:
from leastSquares.helpers import load_data
from leastSquares.least_squares import least_squares
from linearRegression_gradientDescent import costs as cts
# load dataset
x, y = load_data()
print("shape of x {v}".format(v=x.shape))
print("shape of y {v}".format(v=y.shape))

shape of x (50,)
shape of y (50,)


# Cross-Validation and Bias-Variance decomposition
## Cross-Validation
Implementing 4-fold cross-validation below:

In [11]:
def kfold_iter(y, tx, k, seed=1, shuffle=True):
    """Generate a fold iterator for a dataset."""
    np.random.seed(seed)
    
    data_size = len(y)
    
    if shuffle == True:
        shuffled_indices = np.random.permutation(np.arange(data_size))
        shuffled_tx = tx[shuffled_indices]
        shuffled_y = y[shuffled_indices]
    else:
        shuffled_tx = tx
        shuffled_y = y
        
    fold_size = ceil(data_size / k)
    for fold_num in range(k):
        start_index = fold_num * fold_size
        end_index = min((fold_num+1) * fold_size, data_size)
        mask = np.array([i in np.arange(start_index, end_index) for i in range(data_size)])
        data_test = (shuffled_tx[mask], shuffled_y[mask])
        data_train = (shuffled_tx[~mask], shuffled_y[~mask])
        yield data_train, data_test

In [16]:
from leastSquares.build_polynomial import build_poly
from splitData.split import split_data
from ridgeRegression import ridge_regression
from linearRegression_gradientDescent.helpers import batch_iter
from math import ceil

def cross_validation(y, tx, k, lambda_):
    """return the loss of ridge regression."""
    N, D = tx.shape
    
    ws = np.zeros((k, D))
    losses_train = []
    losses_test  = []
    for i, data in enumerate(kfold_iter(y, tx, k)):       
        data_train, data_test = data
        # get train and test data 
        tx_train, y_train = data_train
        tx_test, y_test = data_test
        # find optimal weights (ridge regression)
        loss_train, w_star = ridge_regression(y_train, tx_train, lambda_)
        loss_test = cts.ridge_mse(y_test, tx_test, w_star, lambda_)
        # update
        ws[i, :] = w_star
        losses_train.append(loss_train)
        losses_test.append(loss_test)
    # aggregate weights across each iteration
#     w_star = np.mean(ws, axis=0)
    
    return losses_train, losses_test, ws

In [17]:
degree = 5
k = 4
lambda_ = 2
tx = build_poly(x, degree)

cross_validation(y, tx, k, lambda_)

TypeError: 'module' object is not callable